In [ ]:
# Libraries
library(tidyverse)
library(ggtreeExtra)
library(ggtree)
library(treeio)
library(tidytree)
library(ggstar)
library(ggplot2)
library(ggnewscale)
library(stringr)

In [199]:
# Load data
#----------

## Orthofinder Tree
ogtree <- read.tree("eDNA_Family/eDNA_Family.nwk")

## Sharedness
comparison <- read.table("family_comparison_table.eukaryota.tsv", header=TRUE)
colnames(comparison)[1] <- "label"

In [200]:
# Color palette
tree_data <- as_tibble(ogtree)

color_data <- tree_data %>%
filter(grepl("^k__", label)) %>%
select(node, label)

# General color
color_data$color <- "Undefined"

# Specific colors
color_data$color[color_data$label == "k__Metazoa"] <- "Metazoa"
color_data$color[color_data$label == "k__Viridiplantae"] <- "Viridiplantae"
color_data$color[color_data$label == "k__Fungi"] <- "Fungi"

# Adjust label
color_data$label <- str_replace(color_data$label, "k__","")
color_data$label <- str_replace(color_data$label, "__P\\+","")
color_data$label <- str_replace(color_data$label, "__C\\+","")
color_data$label <- str_replace(color_data$label, "__F\\+","")
color_data$label <- str_replace(color_data$label, "__O\\+","")

In [201]:
# Correct data
#-------------

# Tip labels
ogtree$tip.label <- str_replace(ogtree$tip.label,"f__","")
ogtree$tip.label <- str_replace(ogtree$tip.label,"Astasiidae","Astasiaceae")
ogtree$tip.label <- str_replace(ogtree$tip.label,"Heligmosomidae","Heligmosomatidae")

In [35]:
# Not included families
#missing <-c("Amphidiniaceae","Botrydiopsidaceae","Ciliophryaceae","Spiromastigoidaceae","Lingulodiniaceae","Ophirinidae","Parodontellaceae","Podolampadaceae","Sericomyxidae")
#misslabeled <- comparison$family[!comparison$family %in% ogtree$tip.label]
#misslabeled[!misslabeled %in% missing]

[1] "Archaeobalanidae" "Developea"        "Zoothamniidae"

In [202]:
# Reduce edge length
ogtree$edge.length[ogtree$edge[,2] <= Ntip(ogtree)] <- ogtree$edge.length[ogtree$edge[,2] <= Ntip(ogtree)] * 0.5

In [203]:
# Heatmap data
heatdata <- comparison[comparison$label %in% ogtree$tip.label,]
heatdata$shared <- ifelse(heatdata$belongin == "shared", "Present", "Absent")
heatdata$shotgun <- ifelse(heatdata$belongin == "shotgun", "Present", "Absent")
heatdata$metabarcoding <- ifelse(heatdata$belongin == "metabarcoding", "Present", "Absent")
heatdata <- heatdata[,c(-4)]

# Order
heatdata <- heatdata[match(ogtree$tip.label, heatdata$label),]

# Heatdata
rownames(heatdata) <- ogtree$tip.label
table(rownames(heatdata) == heatdata$label)


TRUE 
1142 

In [8]:
head(heatdata)

,label,metabarcoding,shotgun,shared
,<chr>,<chr>,<chr>,<chr>
Dientamoebidae,Dientamoebidae,Absent,Present,Absent
Asteriidae,Asteriidae,Absent,Absent,Present
Amphiuridae,Amphiuridae,Present,Absent,Absent
Ophiuridae,Ophiuridae,Present,Absent,Absent
Brissidae,Brissidae,Present,Absent,Absent
Echinidae,Echinidae,Present,Absent,Absent


In [210]:
options(repr.plot.width = 25, repr.plot.height = 25, repr.plot.res = 100)

# Base tree
p <- ggtree(ogtree, layout="fan", open.angle = 90, color="black", size=0.25)

# Rotate tree
p <- rotate_tree(p, 90)

# Metabarcoding heatmap
p <- p + new_scale_fill()
p <- gheatmap(p, heatdata[,-c(1,3,4), drop=F], offset=0.3, width=0.05, colnames=TRUE, colnames_angle = 0, hjust=0, font.size = 12) +
scale_fill_manual(values = c("gray90","gold"), na.translate=F) +
guides(fill="none")

# Shotgun heatmap
p <- p + new_scale_fill()
p <- gheatmap(p, heatdata[,-c(1,2,4), drop=F], offset=0.6, width=0.05, colnames=TRUE, colnames_angle = 0, hjust=0, font.size = 12) +
scale_fill_manual(values = c("gray90","dodgerblue"), na.translate=F) +
guides(fill="none")

# Shared heatmap
p <- p + new_scale_fill()
p <- gheatmap(p, heatdata[,-c(1,2,3), drop=F], offset=-0.05, width=0.05, colnames=TRUE, colnames_angle = 0, hjust=0, font.size = 12) +
scale_fill_manual(values = c("gray90","gray60"), na.translate=F) +
guides(fill="none")

# Highlight
p <- p + new_scale_fill()
p <- p + geom_highlight(data = color_data, mapping=aes(node=node, fill=color), alpha=0.5) + 
		 scale_fill_manual(values = c("coral3","darkcyan","rosybrown3","darkolivegreen3")) +
		 labs(fill = "Kingdom")

# Format
p <- p + guides(color = guide_legend(override.aes = list(size=6))) +
theme(legend.position = c(0.62,0.7),
      legend.key.size = unit(2, 'cm'),
      legend.title = element_text(size=46, face="bold"),
      legend.text = element_text(size=50),
      legend.direction = "vertical",
      legend.box = "vertical")

# Save SVG tree and modify by hand
svg(filename = "Fam_Tree.svg", height = 25, width = 25)
p
dev.off()

Scale for y is already present.
Adding another scale for y, which will replace the existing scale.
Coordinate system already present. Adding new coordinate system, which will
replace the existing one.
Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.
Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.
Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.


png 
  2